# Homework 2: Discovery of Frequent Itemsets and Association Rules

# The main tasks in Homework (2) are:

1)	Finding frequent itemsets with support at least s; The key concept in the Apriori algorithm is that it assumes all subsets of a frequent itemset to be frequent. Similarly, for any infrequent itemset, all its supersets must also be infrequent.

2)	Generating association rules with confidence at least c from the itemsets found in the first step. The Apriori algorithm use for this purpose measures such as Support, Confidence,

# The Apriori algorithm

The main steps of the implemented Apriori algorithm using python to solve task 1 and task 2 in this homework are as follows:

1)	Initialize:

  + Read the given sale transaction dataset (which includes generated transactions (a set of baskets).

  + Clean Data

  + Define a minimum support threshold (min_support), indicating the minimum number of transactions an item-set must appear in to be considered frequent.

2)	Generate Frequent Itemsets:

+ Search the given sale transaction dataset and count the ocurrences of each item set.

+ Identify frequent 1-itemsets (individual items) whose support is greater than or equal to the minimum support threshold.

3)	Generate Candidate Itemsets (k > 1):
+ For each subsequent iteration k, generate candidate itemsets of size k based on the frequent itemsets of size k-1 found in the previous step.

+ Search the transactional database and count the support of each candidate itemset, i.e., the number of transactions that contain the itemset.

+ Identify frequent itemsets of size k whose support is greater than or equal to the minimum support threshold.
  + Count Support of Candidate Itemsets:
  + Remove candidate itemsets that have subsets of size k-1 that are not frequent.
+	These become the frequent itemsets for the current iteration

4)	Repeat Until No More Frequent Itemsets:

+ Continue the process of generating candidate itemsets, pruning, counting support, and identifying frequent itemsets until no more frequent itemsets can be found.

5)	Generate Association Rules:

+	Once all frequent itemsets have been discovered, generate association rules from them.
+	For each frequent itemset, generate all possible non-empty subsets, and calculate the confidence of the rule. Retain rules that meet a minimum confidence threshold.

6)	Output:
+	The algorithm outputs the discovered frequent itemsets and association rules that satisfy the specified support and confidence thresholds.


# Intialization

In [ ]:
# Import necessary libraries
import pandas as pd
from itertools import chain, combinations
import os
import glob
import matplotlib.pyplot as plt
from collections import Counter
import statistics
import math

+ Load Data

In [ ]:
# Combine CoLab with my google drive (file path)

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive/Colab Notebooks/DataMining Course 2023/Homework 2"

  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)

  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/My Drive/Colab Notebooks/DataMining Course 2023/Homework 2  exists.

Colab: Changing directory to  /content/gdrive/My Drive/Colab Notebooks/DataMining Course 2023/Homework 2
/content/gdrive/My Drive/Colab Notebooks/DataMining Course 2023/Homework 2


In [ ]:
# Find the current path
current_path = os.getcwd()
print("Current Path:", current_path)

# List all .dat files in subdirectories
dat_files = [os.path.join(root, file) for root, dirs, files in os.walk(current_path) for file in files if file.endswith(".dat")]

# Print the list of .dat files
print("List of .dat files:")
for dat_file in dat_files:
    print(dat_file)

Current Path: /content/gdrive/My Drive/Colab Notebooks/DataMining Course 2023/Homework 2
List of .dat files:
/content/gdrive/My Drive/Colab Notebooks/DataMining Course 2023/Homework 2/Data/T10I4D100K.dat


In [ ]:
# Read Data
# Load all .dat files in the current path

#path_file = "../Data/T10I4D100K.dat"
path_file = glob.glob("*/*.dat")
print (path_file)
dataSets = pd.read_csv(path_file[0], header=None)
print(dataSets)

['Data/T10I4D100K.dat']
                                                       0
0      25 52 164 240 274 328 368 448 538 561 630 687 ...
1                39 120 124 205 401 581 704 814 825 834 
2                        35 249 674 712 733 759 854 950 
3                39 422 449 704 825 857 895 937 954 964 
4      15 229 262 283 294 352 381 708 738 766 853 883...
...                                                  ...
99995                        39 132 623 704 795 825 853 
99996    72 120 241 268 351 354 545 567 606 749 841 893 
99997                            82 313 406 421 438 629 
99998     69 75 140 175 285 494 598 614 641 723 829 871 
99999  32 239 372 419 448 510 540 581 674 752 802 844...

[100000 rows x 1 columns]


In [ ]:
# The head() function is used to display the top rows of a DataFrame. By default, it shows the first five rows.
# Assuming transactions is a DataFrame, it likely represents a dataset where each row corresponds to a transaction (basket),
# and the columns represent items or attributes associated with each transaction
print(dataSets.head())
print("Number of baskets: ", len(dataSets))

                                                   0
0  25 52 164 240 274 328 368 448 538 561 630 687 ...
1            39 120 124 205 401 581 704 814 825 834 
2                    35 249 674 712 733 759 854 950 
3            39 422 449 704 825 857 895 937 954 964 
4  15 229 262 283 294 352 381 708 738 766 853 883...
Number of baskets:  100000


+ Process Data

In [ ]:
# The transactions are stored in the first column of the DataFrame
dataSets_list = dataSets[dataSets.columns[0]]
basket_list = []

for basket_items_str in dataSets_list:
    basket_items = basket_items_str.split()

    # Create a list to store the items in the current basket
    current_basket = []

    for item in basket_items:
        item_int = int(item)  # Convert each item (digit) to an integer
        current_basket.append(item_int)

    # Add the current basket to the list of baskets
    basket_list.append(current_basket)

# Print the 11th basket (index 10)
print(basket_list[0])
print(len(basket_list))
# Now, basket_list contains lists of integers for each row


[25, 52, 164, 240, 274, 328, 368, 448, 538, 561, 630, 687, 730, 775, 825, 834]
100000


+ Define a minimum support threshold (min_support),
 + Indicating the minimum number of transactions an item-set must appear in to be considered frequent.
 + As a rule of thumb, min_support value is 1% of the number of baskets.

# 1) Generate Frequent Itemsets using Aprior Alogroitm with support at least s

+ Generate Frequent Itemsets: dentify frequent 1-itemsets (individual items) whose support is greater than or equal to the minimum support threshold
+  Using Aprior Alogroitm for finding frequent itemsets with support at least s;

In [ ]:
def generate_frequent_1_itemsets(transactions, min_support):
    candidate_itemsets = []
    support_itemsets = []
    itemsets_count = {}

    # Step 1: Count the support of each item (minimum occurrence)
    for transaction in transactions:
        for item in transaction:
            if frozenset([item]) in itemsets_count:
                itemsets_count[frozenset([item])] += 1
            else:
                itemsets_count[frozenset([item])] = 1

    # Step 2: Find frequent items (items with occurrence >= minimum support)
    for itemset, count in itemsets_count.items():
        support = count / len(transactions)
        if support >= min_support:
            candidate_itemsets.append(set(itemset))  # Convert frozenset back to set
            support_itemsets.append(support)  # Convert frozenset back to list
    return candidate_itemsets,support_itemsets


In [ ]:
# Example usage:
min_support_percentage= 0.01 # % 1
#[frequent_1_itemsets, frequency] = generate_frequent_1_itemsets(Data, min_support_percentage)
frequent_1_itemsets_given_dataset,s = generate_frequent_1_itemsets(basket_list, min_support_percentage)
print("Frequent 1-Itemsets:", frequent_1_itemsets_given_dataset)
print("Support_Itemsets:", s)

Frequent 1-Itemsets: [{25}, {52}, {240}, {274}, {368}, {448}, {538}, {561}, {630}, {687}, {775}, {825}, {834}, {39}, {120}, {205}, {401}, {581}, {704}, {814}, {35}, {674}, {733}, {854}, {950}, {422}, {449}, {857}, {895}, {937}, {964}, {229}, {283}, {294}, {381}, {708}, {738}, {766}, {853}, {883}, {966}, {978}, {104}, {143}, {569}, {620}, {798}, {185}, {214}, {350}, {529}, {658}, {682}, {782}, {809}, {947}, {970}, {227}, {390}, {71}, {192}, {208}, {279}, {280}, {496}, {530}, {597}, {618}, {675}, {720}, {914}, {932}, {183}, {217}, {276}, {653}, {706}, {878}, {161}, {175}, {177}, {424}, {490}, {571}, {623}, {795}, {910}, {960}, {125}, {130}, {392}, {461}, {862}, {27}, {78}, {900}, {921}, {147}, {411}, {572}, {579}, {778}, {803}, {266}, {290}, {458}, {523}, {614}, {888}, {944}, {43}, {70}, {204}, {334}, {480}, {513}, {874}, {151}, {504}, {890}, {73}, {310}, {419}, {469}, {722}, {810}, {844}, {846}, {918}, {967}, {326}, {403}, {526}, {774}, {788}, {789}, {975}, {116}, {198}, {201}, {171}, {

Generate Candidate Itemsets (k > 1):

+ For each subsequent iteration k, generate candidate itemsets of size k based on the frequent itemsets of size k-1 found in the previous step.
+	This involves joining pairs of frequent (k-1)-itemsets to form new candidates.

In [ ]:
from itertools import combinations

def generate_candidate_itemsets(prev_itemsets,k):
    candidates = []
    n = len(prev_itemsets)
    for i in range(n):
        for j in range(i + 1, n):
            itemset1 = prev_itemsets[i]
            itemset2 = prev_itemsets[j]
            union_set = itemset1.union(itemset2)

            if len(union_set) == k:
                candidates.append(union_set)

    return candidates

In [ ]:
# Example usage of the generate_candidate_itemsets function

# Given itemsets
itemsets = [{1, 2}, {2, 3}, {3, 4}, {4, 5}]

k = 3
# Generate candidate itemsets
candidate_itemsets = generate_candidate_itemsets(itemsets,k)

# Print the result
print(candidate_itemsets)


[{1, 2, 3}, {2, 3, 4}, {3, 4, 5}]


In [ ]:
# Generate candidate itemsets
# Example usage:
min_support_percentage= 0.01 # % 1
k = 2
frequent_1_itemsets_given_dataset,s = generate_frequent_1_itemsets(basket_list, min_support_percentage)
candidate_itemsets_given_dataset = generate_candidate_itemsets(frequent_1_itemsets_given_dataset, k)

# Print the result
print(candidate_itemsets_given_dataset)


[{25, 52}, {240, 25}, {25, 274}, {368, 25}, {448, 25}, {25, 538}, {25, 561}, {25, 630}, {25, 687}, {25, 775}, {25, 825}, {25, 834}, {25, 39}, {120, 25}, {25, 205}, {25, 401}, {25, 581}, {704, 25}, {25, 814}, {25, 35}, {25, 674}, {25, 733}, {25, 854}, {25, 950}, {25, 422}, {25, 449}, {857, 25}, {25, 895}, {25, 937}, {25, 964}, {25, 229}, {25, 283}, {25, 294}, {25, 381}, {25, 708}, {25, 738}, {25, 766}, {25, 853}, {25, 883}, {25, 966}, {25, 978}, {104, 25}, {25, 143}, {25, 569}, {25, 620}, {25, 798}, {25, 185}, {25, 214}, {25, 350}, {25, 529}, {25, 658}, {25, 682}, {25, 782}, {25, 809}, {25, 947}, {25, 970}, {25, 227}, {25, 390}, {25, 71}, {192, 25}, {208, 25}, {25, 279}, {280, 25}, {496, 25}, {25, 530}, {25, 597}, {25, 618}, {25, 675}, {720, 25}, {25, 914}, {25, 932}, {25, 183}, {25, 217}, {25, 276}, {25, 653}, {25, 706}, {25, 878}, {25, 161}, {25, 175}, {25, 177}, {424, 25}, {25, 490}, {25, 571}, {25, 623}, {25, 795}, {25, 910}, {960, 25}, {25, 125}, {25, 130}, {392, 25}, {25, 461}, {2

In [ ]:
def calculate_support(transactions, candidates_list):
    """
    Calculate support for candidates in transactions.
    """
    count_list = []

    for candidate_set in candidates_list:
        count = 0
        for transaction in transactions:
            if candidate_set.issubset(transaction):
                # If the candidate is a subset of the transaction, increment its count
                count += 1
        count_list.append(count)

    return count_list


In [ ]:
def apriori(transaction_list, min_support_percentage):
    """
    Apriori algorithm to find frequent itemsets.
    """
    # Find frequent 1-itemsets
    frequent_1_itemsets, support_frequent_1_itemsets = generate_frequent_1_itemsets(transaction_list, min_support_percentage)

    # Initialize list to store frequent itemsets
    frequent_itemsets = [frequent_1_itemsets]

    k = 2
    while True:
        # Generate candidate itemsets of size k
        candidate_itemsets = generate_candidate_itemsets(frequent_itemsets[-1], k)

        # Calculate support for each candidate itemset
        support_counts = calculate_support(transaction_list, candidate_itemsets)

        # Prune candidates based on minimum support
        frequent_candidates = []
        for i in range(len(candidate_itemsets)):
            if support_counts[i] > min_support_percentage:
                frequent_candidates.append(candidate_itemsets[i])

        # If no frequent itemsets of size k are found, stop
        if not frequent_candidates:
            break

        # Append frequent itemsets to the list
        frequent_itemsets.append(frequent_candidates)

        k += 1

    return frequent_itemsets


In [ ]:
# Sample transaction list
transaction_list = [[1, 2, 3],
                    [1, 4],
                    [1, 4],
                    [1, 2, 3],
                    [1, 3, 4]
                    ]

# Minimum support threshold (in this example, set to 0.4)
min_support = 0.01

# Using the apriori function
result = apriori(transaction_list, min_support)

# Displaying the frequent itemsets
for k, itemsets in enumerate(result):
    print(f"Frequent {k + 1}-itemsets:")
    for itemset in itemsets:
        print(itemset)
    print()


Frequent 1-itemsets:
{1}
{2}
{3}
{4}

Frequent 2-itemsets:
{1, 2}
{1, 3}
{1, 4}
{2, 3}
{3, 4}

Frequent 3-itemsets:
{1, 2, 3}
{1, 2, 3}
{1, 3, 4}
{1, 2, 3}
{1, 3, 4}
{1, 3, 4}



In [ ]:
# Minimum support threshold (in this example, set to 0.4)
min_support = 0.05

# Using the apriori function
result = apriori(basket_list, min_support)

# Displaying the frequent itemsets
for k, itemsets in enumerate(result):
    print(f"Frequent {k + 1}-itemsets:")
    for itemset in itemsets:
        print(itemset)
    print()

# 2)	Generating association rules with confidence at least c from the itemsets found in the first step.


In [ ]:
from itertools import combinations

def generate_subsets(input_set):
    input_list = list(input_set)
    result = []

    for r in range(1, len(input_list) + 1):
        for subset in combinations(input_list, r):
            result.append(set(subset))

    return result

# Example usage:
input_set = {1, 2, 3}
subsets = generate_subsets(input_set)

print(f"Input Set: {input_set}")
print("All Subsets:")
print(subsets)



Input Set: {1, 2, 3}
All Subsets:
[{1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}, {1, 2, 3}]


In [ ]:
def calculate_support_c(candidate_itemset, transactions):
    count = 0
    for transaction in transactions:
        if set(candidate_itemset).issubset(transaction):
            count += 1
    supprt_c= count/len(transactions)
    return supprt_c

In [ ]:
def calculate_confidence(antecedent, consequent, frequent_itemsets):
    antecedent_union_consequent = antecedent.union(consequent)
    antecedent_support = calculate_support_c(antecedent, frequent_itemsets)
    antecedent_union_consequent_support = calculate_support_c(antecedent_union_consequent, frequent_itemsets)
    confidence = antecedent_union_consequent_support / antecedent_support
    return confidence

In [ ]:
def generate_association_rules(frequent_itemsets, min_confidence):
    association_rules = []

    for consequent_itemset in frequent_itemsets:
        antecedent_itemsets = generate_subsets(consequent_itemset)

        for antecedent in antecedent_itemsets:
            confidence = calculate_confidence(antecedent, consequent_itemset, frequent_itemsets)

            if confidence >= min_confidence:
                association_rules.append((antecedent, consequent_itemset, confidence))

    return association_rules


In [ ]:
# Assume you have the frequent_itemsets generated from your data
frequent_itemsets = [
    {1, 2},
    {2, 3},
    {1, 3},
    {1, 2, 3},
]

# Set your minimum confidence threshold
min_confidence = 0.5

# Call the generate_association_rules function
association_rules = generate_association_rules(frequent_itemsets, min_confidence)

# Print the generated association rules
for rule in association_rules:
    antecedent, consequent, confidence = rule
    print(f"Rule: {antecedent} -> {consequent}, Confidence: {confidence}")


Rule: {1} -> {1, 2}, Confidence: 0.6666666666666666
Rule: {2} -> {1, 2}, Confidence: 0.6666666666666666
Rule: {1, 2} -> {1, 2}, Confidence: 1.0
Rule: {2} -> {2, 3}, Confidence: 0.6666666666666666
Rule: {3} -> {2, 3}, Confidence: 0.6666666666666666
Rule: {2, 3} -> {2, 3}, Confidence: 1.0
Rule: {1} -> {1, 3}, Confidence: 0.6666666666666666
Rule: {3} -> {1, 3}, Confidence: 0.6666666666666666
Rule: {1, 3} -> {1, 3}, Confidence: 1.0
Rule: {1, 2} -> {1, 2, 3}, Confidence: 0.5
Rule: {1, 3} -> {1, 2, 3}, Confidence: 0.5
Rule: {2, 3} -> {1, 2, 3}, Confidence: 0.5
Rule: {1, 2, 3} -> {1, 2, 3}, Confidence: 1.0


# Implementation on the given dataset

In [ ]:
min_support = 0.05 # 5%
min_confidence = 0.6 # 60%

frequent_itemsets_im = apriori(basket_list, min_support)
association_rules_im = generate_association_rules(frequent_itemsets_im, min_confidence)

print("Association Rules:")
for rule in association_rules_im:
    antecedent, consequent, confidence = rule
    print(f"{antecedent} => {consequent} (confidence: {confidence:.2f})")